In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
filename='/content/drive/My Drive/image-data.zip'
with ZipFile(filename,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [2]:
# Importing important libraries
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.applications import ResNet152V2,ResNet50V2,ResNet101V2,DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
!git clone https://github.com/bckenstler/CLR.git
from tensorflow.keras.callbacks import *
from CLR.clr_callback import CyclicLR
import imgaug as ia
from imgaug import augmenters as iaa
#from keras import pyimagesearch
import os

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
#from tensorflow.compat.v1 import ConfigProto
#from tensorflow.compat.v1 import InteractiveSession
#config = ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.2
#config.gpu_options.allow_growth = True

# Create a session with the above options specified.
#session=InteractiveSession(config=config)

# input image dimensions
img_height = 64
img_width = 64
channels = 3

# Hyperparameters
batch_size = 128
num_classes = 200
epochs = 24
num_train = 100000
num_validation = 10000

Cloning into 'CLR'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 252 (delta 1), reused 4 (delta 0), pack-reused 244
Receiving objects: 100% (252/252), 2.06 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (87/87), done.


Using TensorFlow backend.


In [ ]:
val_data = pd.read_csv('/content/image-data/val/val_annotations.txt', sep='\t', header=None, 
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

,File,Class
0,val_0.JPEG,n07753592
1,val_1.JPEG,n02883205
2,val_2.JPEG,n04532670


# Creating Customized Image augumentator

In [3]:
# Defining Customized Imagedatagenerator using imgaug library
def CustomImageDataGen(input_img):
  # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
  # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
  # image.
  sometimes = lambda aug: iaa.Sometimes(0.5, aug)
  
  seq = iaa.Sequential([
      iaa.Fliplr(0.5), # horizontal flips
      iaa.Flipud(0.2), # vertical flips
      
      # Small gaussian blur with random sigma between 0 and 0.5.
      # But we only blur about 50% of all images.
      sometimes(iaa.GaussianBlur(sigma=(0, 2.0))),
      
      # crop images by -10% to 20% of their height/width
      sometimes(iaa.CropAndPad(
          percent=(-0.1, 0.2),
          pad_mode=ia.ALL,
          pad_cval=(0, 255)
        )),
      
      # Apply affine transformations to some of the images
      # - scale to 80-120% of image height/width (each axis independently)
      # - translate by -20 to +20 relative to height/width (per axis)
      # - rotate by -45 to +45 degrees
      # - shear by -16 to +16 degrees
      # - order: use nearest neighbour or bilinear interpolation (fast)
      # - mode: use any available mode to fill newly created pixels
      #         see API or scikit-image for which modes are available
      # - cval: if the mode is constant, then use a random brightness
      #         for the newly created pixels (e.g. sometimes black,
      #         sometimes white)
      sometimes(iaa.Affine(
          scale={"x": (0.8, 1.5), "y": (0.8, 1.5)},
          translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
          rotate=(-45, 45),
          shear=(-16, 16),
          order=[0, 1],
          cval=(0, 255),
          mode=ia.ALL
      )),
      
      #drop 2-5% percent of the original size, leading to large dropped
      # rectangles.
      sometimes(iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    )),
                
      # Make some images brighter and some darker.
      # In 20% of all cases, we sample the multiplier once per channel,
      # which can end up changing the color of the images.
      sometimes(iaa.Multiply((0.8, 1.2), per_channel=0.2)),
      
      #Improve or worsen the contrast of images.
      #Comment it out after third model run (extreme saturation)
      sometimes(iaa.ContrastNormalization((0.75, 1.5), per_channel=0.5)), 
     ],
     # do all of the above augmentations in random order
     random_order = True) # apply augmenters in random order
  
  output_img = seq.augment_image(input_img)
  return output_img

train_datagen = ImageDataGenerator(rescale=1/255., preprocessing_function = CustomImageDataGen)
valid_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = image.ImageDataGenerator(rescale= 1./255)

# Importing training set and validation set

In [ ]:
train_generator = train_datagen.flow_from_directory( '/content/image-data/train/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, seed=101)

Found 90000 images belonging to 200 classes.


In [ ]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='/content/image-data/old_val/images/', 
                                                         x_col='File', y_col='Class', 
                                                         target_size=(img_width, img_height),
                                                         class_mode='categorical', 
                                                         batch_size=batch_size, 
                                                         shuffle=False, seed=101)

Found 10000 validated image filenames belonging to 200 classes.


In [4]:
# Model building
input = Input(shape=(img_height, img_width, channels))
 
# Block 1
layer0 = Conv2D(32, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(input)
layer0 = BatchNormalization()(layer0)
layer0 = Activation('relu')(layer0)
 
skip_connection_1 = layer0
 
# Block 2
 
layer1 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer0)
layer1 = BatchNormalization()(layer1)
layer1 = Activation('relu')(layer1)
 
layer2 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer1)
layer2 = BatchNormalization()(layer2)
layer2 = Activation('relu')(layer2)
 
layer3 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer2)
layer3 = BatchNormalization()(layer3)
layer3 = Activation('relu')(layer3)
 
layer4 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer3)
layer4 = BatchNormalization()(layer4)
layer4 = Activation('relu')(layer4)
 
layer5 = concatenate([skip_connection_1, layer4])
layer5 = BatchNormalization()(layer5)
layer5 = Activation('relu')(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)
 
skip_connection_2 = layer5
 
# Block 3
 
layer6 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer5)
layer6 = BatchNormalization()(layer6)
layer6 = Activation('relu')(layer6)
 
layer7 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer6)
layer7 = BatchNormalization()(layer7)
layer7 = Activation('relu')(layer7)
 
layer8 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer7)
layer8 = BatchNormalization()(layer8)
layer8 = Activation('relu')(layer8)
 
layer9 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer8)
layer9 = BatchNormalization()(layer9)
layer9 = Activation('relu')(layer9)
 
layer10 = concatenate([skip_connection_2, layer9])
layer10 = BatchNormalization()(layer10)
layer10 = Activation('relu')(layer10)
layer10 = MaxPooling2D(pool_size=(2, 2))(layer10)
 
skip_connection_3 = layer10
 
 
# Block 4
 
layer11 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer10)
layer11 = BatchNormalization()(layer11)
layer11 = Activation('relu')(layer11)
 
layer12 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer11)
layer12 = BatchNormalization()(layer12)
layer12 = Activation('relu')(layer12)
 
layer13 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer12)
layer13 = BatchNormalization()(layer13)
layer13 = Activation('relu')(layer13)
 
layer14 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer13)
layer14 = BatchNormalization()(layer14)
layer14 = Activation('relu')(layer14)
 
layer15 = concatenate([skip_connection_3, layer14])
layer15 = BatchNormalization()(layer15)
layer15 = Activation('relu')(layer15)
layer15 = MaxPooling2D(pool_size=(2, 2))(layer15)
 
 
#Layer 16
layer16 = Conv2D(num_classes, (1,1), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=l2(2e-4))(layer15)
layer16 = GlobalAveragePooling2D()(layer16)
 
#Output Layer
output = Activation('softmax')(layer16)

In [5]:
# Model Summary
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
 # Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:
# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath2 = ModelCheckpoint(filepath="/content/drive/My Drive/model1.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=30,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath2]
                   )

In [ ]:
 # The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath2 = '/content/drive/My Drive/model1.hdf5'
model.load_weights(checkpoint_filepath2)

In [ ]:
 # Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:
 
# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath2 = ModelCheckpoint(filepath="/content/drive/My Drive/model2.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=30,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath2]
                   )

In [ ]:
 # The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath2 = '/content/drive/My Drive/model2.hdf5'
model.load_weights(checkpoint_filepath2)

In [ ]:
 # Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:
 
# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath2 = ModelCheckpoint(filepath="/content/drive/My Drive/model3.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=30,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath2]
                   )

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath2 = '/content/drive/My Drive/model3.hdf5'
model.load_weights(checkpoint_filepath2)

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:
 
# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath2 = ModelCheckpoint(filepath="/content/drive/My Drive/model4.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=30,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath2]
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
512/512 [==============================] - ETA: 0s - loss: 3.7516 - accuracy: 0.3890
Epoch 00001: val_accuracy improved from -inf to 0.42140, saving model to /content/drive/My Drive/model4.hdf5
512/512 [==============================] - 1079s 2s/step - loss: 3.7516 - accuracy: 0.3890 - val_loss: 3.5717 - val_accuracy: 0.4214
Epoch 2/30
512/512 [==============================] - ETA: 0s - loss: 3.8149 - accuracy: 0.3785
Epoch 00002: val_accuracy did not improve from 0.42140
512/512 [==============================] - 1073s 2s/step - loss: 3.8149 - accuracy: 0.3785 - val_loss: 3.6297 - val_accuracy: 0.4141
Epoch 3/30
512/512 [==============================] - ETA: 0s - loss: 3.9241 - accuracy: 0.3666
Epoch 00003: val_accuracy did not improve from 0.42140
512/512 [==============================] - 1078s 2s/step - loss: 3.9241 - accuracy: 0.3666 - val_loss: 3.8781 - val_accuracy: 0.3770
Epoch 4/30
512/512

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath3 = '/content/drive/My Drive/model4.hdf5'
model.load_weights(checkpoint_filepath3)

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:

# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath3 = ModelCheckpoint(filepath="/content/drive/My Drive/model5.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=40,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath3]
                   )

Epoch 1/30
512/512 [==============================] - ETA: 0s - loss: 4.7945 - accuracy: 0.4814
Epoch 00001: val_accuracy improved from -inf to 0.51750, saving model to /content/drive/My Drive/model5.hdf5
512/512 [==============================] - 615s 1s/step - loss: 4.7945 - accuracy: 0.4814 - val_loss: 4.6135 - val_accuracy: 0.5175
Epoch 2/30
512/512 [==============================] - ETA: 0s - loss: 4.8789 - accuracy: 0.4685
Epoch 00002: val_accuracy did not improve from 0.51750
512/512 [==============================] - 609s 1s/step - loss: 4.8789 - accuracy: 0.4685 - val_loss: 4.8093 - val_accuracy: 0.4839
Epoch 3/30
512/512 [==============================] - ETA: 0s - loss: 4.9919 - accuracy: 0.4567
Epoch 00003: val_accuracy did not improve from 0.51750
512/512 [==============================] - 609s 1s/step - loss: 4.9919 - accuracy: 0.4567 - val_loss: 5.0111 - val_accuracy: 0.4558
Epoch 4/30
512/512 [==============================] - ETA: 0s - loss: 5.1281 - accuracy: 0.4462
E

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath3 = '/content/drive/My Drive/model5.hdf5'
model.load_weights(checkpoint_filepath3)

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:

# Callbacks
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath3 = ModelCheckpoint(filepath="/content/drive/My Drive/model6.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=40,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath3]
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
512/512 [==============================] - ETA: 0s - loss: 5.9697 - accuracy: 0.5260
Epoch 00001: val_accuracy improved from -inf to 0.54780, saving model to /content/drive/My Drive/model6.hdf5
512/512 [==============================] - 713s 1s/step - loss: 5.9697 - accuracy: 0.5260 - val_loss: 5.9134 - val_accuracy: 0.5478
Epoch 2/40
512/512 [==============================] - ETA: 0s - loss: 6.0388 - accuracy: 0.5172
Epoch 00002: val_accuracy did not improve from 0.54780
512/512 [==============================] - 718s 1s/step - loss: 6.0388 - accuracy: 0.5172 - val_loss: 6.1152 - val_accuracy: 0.5138
Epoch 3/40
512/512 [==============================] - ETA: 0s - loss: 6.1378 - accuracy: 0.5033
Epoch 00003: val_accuracy did not improve from 0.54780
512/512 [==============================] - 724s 1s/step - loss: 6.1378 - accuracy: 0.5033 - val_loss: 6.1236 - val_accuracy: 0.5138
Epoch 4/40
512/512 [=

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath3 = '/content/drive/My Drive/model6.hdf5'
model.load_weights(checkpoint_filepath3)

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:

# Callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath3 = ModelCheckpoint(filepath="/content/drive/My Drive/model7.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=35,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath3]
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/35
512/512 [==============================] - ETA: 0s - loss: 7.1195 - accuracy: 0.6017
Epoch 00001: val_accuracy improved from -inf to 0.61560, saving model to /content/drive/My Drive/model7.hdf5
512/512 [==============================] - 724s 1s/step - loss: 7.1195 - accuracy: 0.6017 - val_loss: 7.0579 - val_accuracy: 0.6156
Epoch 2/35
512/512 [==============================] - ETA: 0s - loss: 7.0800 - accuracy: 0.6120
Epoch 00002: val_accuracy improved from 0.61560 to 0.61760, saving model to /content/drive/My Drive/model7.hdf5
512/512 [==============================] - 717s 1s/step - loss: 7.0800 - accuracy: 0.6120 - val_loss: 7.0598 - val_accuracy: 0.6176
Epoch 3/35
512/512 [==============================] - ETA: 0s - loss: 7.0816 - accuracy: 0.6135
Epoch 00003: val_accuracy did not improve from 0.61760
512/512 [==============================] - 711s 1s/step - loss: 7.0816 - accuracy: 0.6135 - val_

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath3 = '/content/drive/My Drive/model7.hdf5'
model.load_weights(checkpoint_filepath3)

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              #optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:

# Callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath3 = ModelCheckpoint(filepath="/content/drive/My Drive/model8.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=35,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath3]
                   )

Epoch 1/35
512/512 [==============================] - ETA: 0s - loss: 7.0981 - accuracy: 0.6536
Epoch 00001: val_accuracy improved from -inf to 0.62800, saving model to /content/drive/My Drive/model8.hdf5
512/512 [==============================] - 749s 1s/step - loss: 7.0981 - accuracy: 0.6536 - val_loss: 7.2061 - val_accuracy: 0.6280
Epoch 2/35
512/512 [==============================] - ETA: 0s - loss: 7.0859 - accuracy: 0.6513
Epoch 00002: val_accuracy did not improve from 0.62800
512/512 [==============================] - 750s 1s/step - loss: 7.0859 - accuracy: 0.6513 - val_loss: 7.2100 - val_accuracy: 0.6232
Epoch 3/35
512/512 [==============================] - ETA: 0s - loss: 7.0819 - accuracy: 0.6500
Epoch 00003: val_accuracy did not improve from 0.62800
512/512 [==============================] - 752s 1s/step - loss: 7.0819 - accuracy: 0.6500 - val_loss: 7.1984 - val_accuracy: 0.6208
Epoch 4/35
512/512 [==============================] - ETA: 0s - loss: 7.0559 - accuracy: 0.6502
E

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath3 = '/content/drive/My Drive/model8.hdf5'
model.load_weights(checkpoint_filepath3)

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:

# Callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath3 = ModelCheckpoint(filepath="/content/drive/My Drive/model9.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=35,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath3]
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/35
512/512 [==============================] - ETA: 0s - loss: 6.3783 - accuracy: 0.6785
Epoch 00001: val_accuracy improved from -inf to 0.62870, saving model to /content/drive/My Drive/model9.hdf5
512/512 [==============================] - 1094s 2s/step - loss: 6.3783 - accuracy: 0.6785 - val_loss: 6.6074 - val_accuracy: 0.6287
Epoch 2/35
512/512 [==============================] - ETA: 0s - loss: 6.3833 - accuracy: 0.6793
Epoch 00002: val_accuracy did not improve from 0.62870
512/512 [==============================] - 1078s 2s/step - loss: 6.3833 - accuracy: 0.6793 - val_loss: 6.6182 - val_accuracy: 0.6282
Epoch 3/35
512/512 [==============================] - ETA: 0s - loss: 6.4061 - accuracy: 0.6740
Epoch 00003: val_accuracy did not improve from 0.62870
512/512 [==============================] - 1083s 2s/step - loss: 6.4061 - accuracy: 0.6740 - val_loss: 6.6365 - val_accuracy: 0.6233
Epoch 4/35
512/512

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
checkpoint_filepath3 = '/content/drive/My Drive/model9.hdf5'
model.load_weights(checkpoint_filepath3)

In [ ]:
# Compile the Model
model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              #optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              #optimizer = SGD(momentum=0.9),
              metrics=['accuracy'])

In [ ]:
 
# Callbacks
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpoint_filepath3 = ModelCheckpoint(filepath="/content/drive/My Drive/model10.hdf5", save_weights_only=True,verbose=1,
                          monitor='val_accuracy',mode='max',save_best_only=True)
# Fit the Model
model.fit_generator(train_generator,
                    epochs=35,
                    steps_per_epoch= 512,
                    validation_steps= len(validation_generator),
                    validation_data=validation_generator,
                    verbose=1, callbacks=[clr, checkpoint_filepath3]
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/35
 28/512 [>.............................] - ETA: 13:31:43 - loss: 6.3736 - accuracy: 0.6819

In [ ]:
test_set=test_datagen.flow_from_directory('/content/image-data/test',
                                               batch_size = 200,target_size=(64, 64),
                                               shuffle=False,class_mode=None)
test_set.reset()

Found 10000 images belonging to 1 classes.


In [ ]:
checkpoint_filepath3 = '/content/drive/My Drive/model10.hdf5'
model.load_weights(checkpoint_filepath3)

In [ ]:
cnn=model.predict_generator(test_set,verbose=1,steps=len(test_set))

Instructions for updating:
Please use Model.predict, which supports generators.
50/50 [==============================] - 2062s 41s/step


In [ ]:
predicted_class_indices=np.argmax(cnn,axis=1)
print(predicted_class_indices)

[107  40  71 ...   9   1  61]


In [ ]:
labels = (train_generator.class_indices)
print(labels.items())
labels = dict((v,k) for k,v in labels.items())

dict_items([('n01443537', 0), ('n01629819', 1), ('n01641577', 2), ('n01644900', 3), ('n01698640', 4), ('n01742172', 5), ('n01768244', 6), ('n01770393', 7), ('n01774384', 8), ('n01774750', 9), ('n01784675', 10), ('n01855672', 11), ('n01882714', 12), ('n01910747', 13), ('n01917289', 14), ('n01944390', 15), ('n01945685', 16), ('n01950731', 17), ('n01983481', 18), ('n01984695', 19), ('n02002724', 20), ('n02056570', 21), ('n02058221', 22), ('n02074367', 23), ('n02085620', 24), ('n02094433', 25), ('n02099601', 26), ('n02099712', 27), ('n02106662', 28), ('n02113799', 29), ('n02123045', 30), ('n02123394', 31), ('n02124075', 32), ('n02125311', 33), ('n02129165', 34), ('n02132136', 35), ('n02165456', 36), ('n02190166', 37), ('n02206856', 38), ('n02226429', 39), ('n02231487', 40), ('n02233338', 41), ('n02236044', 42), ('n02268443', 43), ('n02279972', 44), ('n02281406', 45), ('n02321529', 46), ('n02364673', 47), ('n02395406', 48), ('n02403003', 49), ('n02410509', 50), ('n02415577', 51), ('n0242302

In [ ]:
predictions = [labels[k] for k in predicted_class_indices]
print(predictions)

['n03444034', 'n02231487', 'n02814860', 'n03026506', 'n03400231', 'n04023962', 'n02699494', 'n09332890', 'n02281406', 'n03250847', 'n02769748', 'n03444034', 'n01774384', 'n01855672', 'n02226429', 'n02415577', 'n02074367', 'n04501370', 'n02892201', 'n09246464', 'n02963159', 'n02793495', 'n02099712', 'n04532670', 'n09332890', 'n02206856', 'n02814860', 'n02132136', 'n02808440', 'n02415577', 'n02948072', 'n03400231', 'n07875152', 'n02132136', 'n03637318', 'n03983396', 'n03937543', 'n03670208', 'n02058221', 'n02423022', 'n03983396', 'n04486054', 'n03160309', 'n02802426', 'n03250847', 'n01950731', 'n04487081', 'n03584254', 'n04597913', 'n07614500', 'n07579787', 'n01742172', 'n09246464', 'n02226429', 'n02231487', 'n04074963', 'n03891332', 'n02410509', 'n02165456', 'n03637318', 'n04501370', 'n01855672', 'n01910747', 'n01774750', 'n04456115', 'n01742172', 'n02699494', 'n04596742', 'n04465501', 'n09193705', 'n03980874', 'n02124075', 'n03447447', 'n07871810', 'n01742172', 'n02437312', 'n03400231'

In [ ]:
filenames=test_set.filenames
print(filenames)
for i in range(0,len(filenames)):
  if filenames[i].startswith('images/'):
    filenames[i]=filenames[i][7:]
print(filenames)

['images/test_0.JPEG', 'images/test_1.JPEG', 'images/test_10.JPEG', 'images/test_100.JPEG', 'images/test_1000.JPEG', 'images/test_1001.JPEG', 'images/test_1002.JPEG', 'images/test_1003.JPEG', 'images/test_1004.JPEG', 'images/test_1005.JPEG', 'images/test_1006.JPEG', 'images/test_1007.JPEG', 'images/test_1008.JPEG', 'images/test_1009.JPEG', 'images/test_101.JPEG', 'images/test_1010.JPEG', 'images/test_1011.JPEG', 'images/test_1012.JPEG', 'images/test_1013.JPEG', 'images/test_1014.JPEG', 'images/test_1015.JPEG', 'images/test_1016.JPEG', 'images/test_1017.JPEG', 'images/test_1018.JPEG', 'images/test_1019.JPEG', 'images/test_102.JPEG', 'images/test_1020.JPEG', 'images/test_1021.JPEG', 'images/test_1022.JPEG', 'images/test_1023.JPEG', 'images/test_1024.JPEG', 'images/test_1025.JPEG', 'images/test_1026.JPEG', 'images/test_1027.JPEG', 'images/test_1028.JPEG', 'images/test_1029.JPEG', 'images/test_103.JPEG', 'images/test_1030.JPEG', 'images/test_1031.JPEG', 'images/test_1032.JPEG', 'images/tes

In [ ]:
f=open("result.txt",'w')
f.write("file_name,category")
f.write("\n")
for i in range(0,len(filenames)):
  f.write(f"{filenames[i]},{predictions[i]}")
  f.write("\n")
f.close()